In [3]:
# libraries
import numpy as np
import pandas as pd
import xgboost as xgb
import pickle
from datetime import datetime

class Inference():
    def __init__(self,data_version,model_version):
        self.data_version=data_version
        self.load_path='intermediate_data/preprocessed_spec_for_inference_{}.pickle'.format(self.data_version)
        self.model_version=model_version
        self.model_path='model/model_xgb_{}.pickle'.format(self.model_version)
        self.save_path='result/result_{}.pickle'.format(self.data_version)
        
    def load_prep_spec(self):
        with open(self.load_path,'rb') as f:
            spec=pickle.load(f)
        return spec

    def load_model(self):
        with open(self.model_path,'rb') as f:
            model=pickle.load(f)
        return model
    
    def inference(self,spec):
        #データの前処理
        X = spec.drop(['price','詳細情報'], axis=1).as_matrix()
        y = spec['price']
        indices=spec.index

        #モデルのロード
        xgb=self.load_model()

        #推論
        prediction=xgb.predict(X)
        
        #結果の比較
        error=y-prediction
        error_per=abs(error)/y*100
        result=pd.DataFrame({
                            'prediction':prediction,
                            'error':error,
                            'error_percent':error_per,
                            },index=indices)
        result_all=pd.concat([spec,result],axis=1)
        treasure=result_all[(result_all['error_percent']>5)&(result_all['error']<0)]
        
#         #結果を出力
        print(result_all)
        return treasure
 
    def save_treasure(self,treasure):
#         print(self.save_path)
        with open(self.save_path,'wb') as f:
            pickle.dump(treasure, f)

if __name__ == '__main__':
    data_version='original'
    model_version='original'
    inf=Inference(data_version,model_version)
    spec=inf.load_prep_spec()
    treasure=inf.inference(spec)
    inf.save_treasure(treasure)

         price      詳細情報     平米単価    専有面積  間取り  築年数  構造  階数  向き  敷地の権利形態  \
0       4480.0  15008384   617600   72.55   17   18   3   7   5        2   
1        690.0  13427251   133700   51.61    9   51   1   4   4        2   
2       1480.0  13259710   132200  111.96   25   36   1   1   4        2   
3       3480.0  13964101   483400      72   17   35   3  12   4        2   
4       3290.0  15085882   991600   33.18    4   21   1   3   0        2   
5       4350.0  14536203   643200   67.64    9   12   1   8   7        2   
6       7980.0  08039636   738300  108.09   17   32   1   3   3        2   
7       5980.0  14282015   844200   70.84    9    9   1   5   5        2   
8       3186.0  11566236   770400   41.36    2   40   3   3   3        2   
9       4280.0  14572746   557100   76.83   17   21   1   4   3        2   
10      6800.0  14953262   902300   75.37   17   10   1  16   5        2   
11     29800.0  14682569  3508800   84.93   17    0   1  20   1        2   
12      4280